In [ ]:
import os
import shutil
import subprocess

INPUT_DIR = "Downloaded_Universities"
PROCESSED_DIR = os.path.join(INPUT_DIR, "processed_csvs")
os.makedirs(PROCESSED_DIR, exist_ok=True)


def run_spider_on_csvs():
    csv_files = [f for f in os.listdir(
        INPUT_DIR) if f.lower().endswith(".csv")]

    for csv_file in csv_files:
        csv_path = os.path.join(INPUT_DIR, csv_file)
        print(f"🚀 Running spider for: {csv_file}")

        try:
            subprocess.run([
                "python", "-m", "scrapy", "crawl", "courses", f"-a", f"csv_file={csv_path}"
            ], check=True)

            # ✅ Move processed CSV immediately
            shutil.move(csv_path, os.path.join(PROCESSED_DIR, csv_file))
            print(f"✅ Finished and moved: {csv_file}")

        except subprocess.CalledProcessError as e:
            print(f"❌ Spider failed for {csv_file}: {e}")


🚀 Running spider for: Abertay University.csv
